<b>Preparing Apple Health data for analysis/visualization<b/>
- Download personal data from app; unzip XML file
- Parse to DataFrame using python standard libraries + Pandas
- Anonymize Personal Identifiable Information (PII) using regular expression replacements
- Explore and wrangle/clean, extracting data related to 'step count'
- Save data of interest to csv for external tabular analysis

In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
import io

In [2]:
#Parse XML content
tree = ET.parse('export.xml')
root = tree.getroot()

#Extract 'Record' log elements
records = []
for record in root.findall('Record'):
    record_data = record.attrib
    records.append(record_data)

#Create DataFrame
df_export = pd.DataFrame(records)
print(df_export.shape)

(154516, 9)


In [3]:
#Pre-display cleaning/wrangling
 #Anonymize Health Kit Device object identifiers
df_export['device'] = df_export['device'].str.replace('<<HKDevice: .*?>, ', 'HKDevice: anonymized>, ', regex=True)
 #Anonymize actual device name
df_export['sourceName'] = df_export['sourceName'].str.replace('.*iPhone.*', 'iPhone', regex=True)

print(df_export.sourceName.value_counts())

 #Remove records from 'Health' source
df_export = df_export[df_export['sourceName'] != 'Health']
df_export.reset_index(drop=True, inplace=True)

df_export.head(2)

sourceName
iPhone    154514
Health         2
Name: count, dtype: int64


,type,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,device
0,HKQuantityTypeIdentifierStepCount,iPhone,15.3.1,count,2022-04-23 16:51:21 -0400,2022-04-23 16:36:00 -0400,2022-04-23 16:45:57 -0400,141,"HKDevice: anonymized>, name:iPhone, manufactur..."
1,HKQuantityTypeIdentifierStepCount,iPhone,15.3.1,count,2022-04-23 16:59:28 -0400,2022-04-23 16:46:25 -0400,2022-04-23 16:49:23 -0400,86,"HKDevice: anonymized>, name:iPhone, manufactur..."


In [4]:
#Cleaning/wrangling continued
 #Simplify 'type' column
df_export['type'] = df_export['type'].str.replace('HKQuantityTypeIdentifier', '')
df_export.head(2)

,type,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,device
0,StepCount,iPhone,15.3.1,count,2022-04-23 16:51:21 -0400,2022-04-23 16:36:00 -0400,2022-04-23 16:45:57 -0400,141,"HKDevice: anonymized>, name:iPhone, manufactur..."
1,StepCount,iPhone,15.3.1,count,2022-04-23 16:59:28 -0400,2022-04-23 16:46:25 -0400,2022-04-23 16:49:23 -0400,86,"HKDevice: anonymized>, name:iPhone, manufactur..."


In [5]:
#Review record categories
df_export.type.value_counts()

type
BasalEnergyBurned                 33074
StepCount                         17134
DistanceWalkingRunning            17129
WalkingSpeed                      16297
WalkingStepLength                 16297
HeadphoneAudioExposure            15565
ActiveEnergyBurned                14467
WalkingDoubleSupportPercentage    14446
WalkingAsymmetryPercentage         7647
FlightsClimbed                     2405
AppleWalkingSteadiness               53
Name: count, dtype: int64

In [6]:
#New df: steps data only
df_steps = df_export[df_export['type']=='StepCount'].copy()
print(df_steps.shape)
df_steps.head(2)

(17134, 9)


,type,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,device
0,StepCount,iPhone,15.3.1,count,2022-04-23 16:51:21 -0400,2022-04-23 16:36:00 -0400,2022-04-23 16:45:57 -0400,141,"HKDevice: anonymized>, name:iPhone, manufactur..."
1,StepCount,iPhone,15.3.1,count,2022-04-23 16:59:28 -0400,2022-04-23 16:46:25 -0400,2022-04-23 16:49:23 -0400,86,"HKDevice: anonymized>, name:iPhone, manufactur..."


In [7]:
#Check dtypes
df_steps.dtypes

type             object
sourceName       object
sourceVersion    object
unit             object
creationDate     object
startDate        object
endDate          object
value            object
device           object
dtype: object

In [8]:
#Convert step count to int + rename
df_steps['stepCount'] = pd.to_numeric(df_steps['value'])
df_steps.stepCount.dtypes

dtype('int64')

In [9]:
#Inspect full 'device' column value for use
print(df_steps.device.values[1])

HKDevice: anonymized>, name:iPhone, manufacturer:Apple Inc., model:iPhone, hardware:iPhone14,5, software:15.3.1>


In [10]:
#Drop unused columns
df_steps.drop(df_steps[['sourceName','sourceVersion','unit','creationDate','device','value']], axis=1, inplace=True)
df_steps.head(2)

,type,startDate,endDate,stepCount
0,StepCount,2022-04-23 16:36:00 -0400,2022-04-23 16:45:57 -0400,141
1,StepCount,2022-04-23 16:46:25 -0400,2022-04-23 16:49:23 -0400,86


In [11]:
#Output startDate range; review datetime formatting
 #ISO 8601 format
print(df_steps.startDate.min())
print(df_steps.startDate.max())

2022-04-23 16:36:00 -0400
2024-06-07 19:59:47 -0400


In [12]:
 #Date column(s) currently string type:
df_steps['startDate'][0]

'2022-04-23 16:36:00 -0400'

In [13]:
#New df: convert date columns to datetime, split values into new columns

df_steps_dt = df_steps.copy()

 #In place
#df_steps_dt['startDate'] = pd.to_datetime(df_steps_dt['startDate'])
#df_steps_dt['startDate'] = pd.to_datetime(df_steps_dt['endDate'])
 #New variables
df_steps_dt['startDatetime'] = pd.to_datetime(df_steps_dt['startDate'])
df_steps_dt['endDatetime'] = pd.to_datetime(df_steps_dt['endDate'])

#Output value
print(df_steps_dt['startDatetime'][0])
 #Verify object type
df_steps_dt['startDatetime'][0]

2022-04-23 16:36:00-04:00


Timestamp('2022-04-23 16:36:00-0400', tz='UTC-04:00')

In [14]:
#Preview data
df_steps_dt.head(2)

,type,startDate,endDate,stepCount,startDatetime,endDatetime
0,StepCount,2022-04-23 16:36:00 -0400,2022-04-23 16:45:57 -0400,141,2022-04-23 16:36:00-04:00,2022-04-23 16:45:57-04:00
1,StepCount,2022-04-23 16:46:25 -0400,2022-04-23 16:49:23 -0400,86,2022-04-23 16:46:25-04:00,2022-04-23 16:49:23-04:00


In [15]:
#Create columns at 'day' and 'time' granularities
df_steps_dt['startDay'], df_steps_dt['startTime'] = df_steps_dt['startDatetime'].dt.date,\
                                                    df_steps_dt['startDatetime'].dt.time

df_steps_dt['endDay'], df_steps_dt['endTime'] = df_steps_dt['endDatetime'].dt.date,\
                                                df_steps_dt['endDatetime'].dt.time

 #Drop + reorder columns
df_steps_dt = df_steps_dt.reindex(columns=['type','startDatetime','startDay','startTime','endDatetime','endDay','endTime','stepCount'])

df_steps_dt.head(2)

,type,startDatetime,startDay,startTime,endDatetime,endDay,endTime,stepCount
0,StepCount,2022-04-23 16:36:00-04:00,2022-04-23,16:36:00,2022-04-23 16:45:57-04:00,2022-04-23,16:45:57,141
1,StepCount,2022-04-23 16:46:25-04:00,2022-04-23,16:46:25,2022-04-23 16:49:23-04:00,2022-04-23,16:49:23,86


In [16]:
#Data quality check
 #Missing values: None
df_steps_dt.isna().sum()

type             0
startDatetime    0
startDay         0
startTime        0
endDatetime      0
endDay           0
endTime          0
stepCount        0
dtype: int64

In [17]:
#Save as csv
df_steps_dt.to_csv('steps_data.csv', index=True)